## Build a Wflow sediment model

This notebook demonstrates how to prepare a **Wflow sediment** model using the command line interace (CLI).

All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

### hydroMT CLI build interface

Lets first check if the Wflow sediment model is recognized by hydromt

In [1]:
# this should return "wflow, wflow_sediment"
!hydromt --models

hydroMT model plugins: wflow (v0.1.4.dev0), wflow_sediment (v0.1.4.dev0)


Using the **hydromt build** API we can setup a complete model from scratch. Let's get an overview of all the available options:

In [2]:
!hydromt build --help

Usage: hydromt build [OPTIONS] MODEL MODEL_ROOT REGION

  Build models from source data.

  Example usage:
  --------------

  To build a wflow model for a subbasin using and point coordinates snapped to cells with stream order >= 4
  hydromt build wflow /path/to/model_root "{'subbasin': [-7.24, 62.09], 'strord': 4}" -i /path/to/wflow_config.ini

  To build a wflow model based on basin ID
  hydromt build wflow /path/to/model_root "{'basin': 230001006}"

  To build a sfincs model based on a bbox (for Texel)
  hydromt build sfincs /path/to/model_root "{'bbox': [4.6891,52.9750,4.9576,53.1994]}"

Options:
  -r, --res FLOAT             Model resolution in model src.
  --build-base / --build-all  Deprecated!
  --opt TEXT                  Component specific keyword arguments, see the
                              setup_<component> method of the specific model
                              for more information about the arguments.
  -i, --config PATH           Path to hydroMT configuration fil

### Model setup configuration

The ini-file contains the model setup configuration and determines which components are build and in which order and optionally sets non-default arguments for each component. This configuration is passed to hydromt using `-i <path_to_ini_file>`. We have prepared several example ini-files which are available in the model repository [examples folder](https://github.com/Deltares/hydromt_wflow/tree/main/examples) and from the [docs(build_configuration)](https://deltares.github.io/hydromt_wflow/latest/user_guide/wflow/build_configuration.html). 

Each header as shown between `[...]` (e.g. `[setup_basemaps]`) corresponds to a model component. All model components are explained in the [docs(model_components)](https://deltares.github.io/hydromt_wflow/latest/user_guide/wflow/components.html). 

We will load the default wflow sediment build ini file for inspection:

In [3]:
fn_ini = "wflow_sediment_build.ini"
with open(fn_ini, "r") as f:
    txt = f.read()
print(txt)

[global]
data_libs        = []               # add optional paths to data yml files

[setup_config]                      # options parsed to wflow ini file <section>.<option>
starttime = 2010-01-01T00:00:00
endtime = 2010-03-31T00:00:00
timestepsecs = 86400

[setup_basemaps]
hydrography_fn   = merit_hydro      # source hydrography data {merit_hydro, merit_hydro_1k}
basin_index_fn   = merit_hydro_index # source of basin index corresponding to hydrography_fn
upscale_method   = ihu              # upscaling method for flow direction data, by default 'ihu'

[setup_rivers]
hydrography_fn   = merit_hydro      # source hydrography data, should correspond to hydrography_fn in setup_basemaps
river_geom_fn    = rivers_lin2019_v1 # river source data with river width and bankfull discharge
river_upa        = 30               # minimum upstream area threshold for the river map [km2]
rivdph_method    = powlaw           # method to estimate depth {'powlaw', 'manning', 'gvf'}
min_rivdph       = 1      

### Setup wflow sediment model from scratch

In [4]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt build wflow_sediment "./wflow_test_sediment" "{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}" -r 0.00833 -i wflow_sediment_build.ini -vv

2022-01-27 12:40:14,167 - build - log - DEBUG - Writing log messages to new file /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_sediment/hydromt.log.
2022-01-27 12:40:14,167 - build - log - INFO - HydroMT version: 0.4.5.dev
2022-01-27 12:40:14,167 - build - main - INFO - Building instance of wflow_sediment model at /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_sediment.
2022-01-27 12:40:14,167 - build - main - INFO - User settings:
2022-01-27 12:40:14,175 - build - model_plugins - DEBUG - Loaded model plugin 'wflow_sediment = hydromt_wflow.wflow_sediment.WflowSedimentModel' (0.1.4.dev0)
2022-01-27 12:40:14,175 - build - model_api - INFO - Initializing wflow_sediment model from hydromt_wflow (v0.1.4.dev0).
2022-01-27 12:40:14,175 - build - model_api - DEBUG - Setting model config options.
2022-01-27 12:40:14,177 - build - model_api - DEBUG - Default config read from /usr/share/miniconda/envs/hydromt-wflow/lib/python3.9/s

2022-01-27 12:40:14,237 - build - data_adapter - INFO - DataCatalog: Getting merit_hydro RasterDataset raster data from /home/runner/.hydromt_data/data/v0.0.6/merit_hydro/{variable}.tif


2022-01-27 12:40:14,306 - build - basin_mask - DEBUG - Parsed region (kind=subbasin): {'strord': 4, 'bounds': [11.7, 45.35, 12.95, 46.7], 'xy': [12.2051, 45.8331]}


2022-01-27 12:40:17,609 - build - basin_mask - INFO - subbasin bbox: [11.7750, 45.8042, 12.7450, 46.6900]


2022-01-27 12:40:17,680 - build - wflow - DEBUG - Adding basins vector to staticgeoms.
2022-01-27 12:40:17,680 - build - basemaps - DEBUG - Mask in dataset assumed to represent subbasins.
2022-01-27 12:40:17,681 - build - basemaps - DEBUG - (Sub)basin at original resolution has 664756 cells.


...

2022-01-27 12:40:43,054 - build - soilgrids - INFO - Interpolate NAN values for PercentOC


2022-01-27 12:40:43,103 - build - soilgrids - INFO - Interpolate NAN values for ErosK
2022-01-27 12:40:43,143 - build - soilgrids - INFO - Interpolate NAN values for USLE_K


2022-01-27 12:40:43,154 - build - model_api - INFO - setup_hydrodem.elevtn_map: wflow_dem
2022-01-27 12:40:43,154 - build - model_api - INFO - setup_hydrodem.river_routing: local-inertial
2022-01-27 12:40:43,154 - build - model_api - INFO - setup_hydrodem.land_routing: kinematic-wave
2022-01-27 12:40:43,154 - build - wflow - INFO - Preparing hydrodem_avg map for routing.
2022-01-27 12:40:43,181 - build - flw - INFO - Condition elevation to D8 flow directions.


2022-01-27 12:40:46,633 - build - wflow - DEBUG - Update wflow config model.river_routing="local-inertial"
2022-01-27 12:40:46,633 - build - wflow - DEBUG - Update wflow config model.land_routing="kinematic-wave"
2022-01-27 12:40:46,633 - build - main - ERROR - 'land'
Traceback (most recent call last):
  File "/usr/share/miniconda/envs/hydromt-wflow/lib/python3.9/site-packages/hydromt/cli/main.py", line 196, in build
    mod.build(region, res, opt=opt)
  File "/usr/share/miniconda/envs/hydromt-wflow/lib/python3.9/site-packages/hydromt/models/model_api.py", line 156, in build
    self._run_log_method(method, **opt[method])
  File "/usr/share/miniconda/envs/hydromt-wflow/lib/python3.9/site-packages/hydromt/models/model_api.py", line 98, in _run_log_method
    return func(*args, **kwargs)
  File "/usr/share/miniconda/envs/hydromt-wflow/lib/python3.9/site-packages/hydromt_wflow/wflow.py", line 469, in setup_hydrodem
    self.config["state"]["lateral"]["land"].pop("qx", None)
KeyError: 'lan

    return func(*args, **kwargs)
  File "/usr/share/miniconda/envs/hydromt-wflow/lib/python3.9/site-packages/hydromt_wflow/wflow.py", line 469, in setup_hydrodem
    self.config["state"]["lateral"]["land"].pop("qx", None)
KeyError: 'land'


The example above means the following: run **hydromt build** with:

* `wflow_sediment` : i.e. build a wflow sediment model
* `./wflow_test_sediment` : output model folder
* `"{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}"` : derive a subbasin with its outlet at the given x,y coordinates and within the given bounds [xmin, ymin, xmax, ymax] (WGS84) snapped to a river with minimum stream order (strord) of 4. All *REGION* options are described in the [docs](https://deltares.github.io/hydromt/latest/user_guide/cli.html#region-options)
* `-r 0.0083333` : build the model at a 30 arc sec (~1km) resolution
* `-i wflow_sediment_build.ini` : setup configuration file using
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.


Next we check which files have been created. The model root should contain two netcdf (.nc) files, one for the staticmaps and one for the forcing data, a wflow configuration (.toml) file. These files are sufficient to run the wflow sediment model. In addition several geometry (.geojson) files are saved in the *staticgeoms* folder.  Finally, the setup process is logged in the hydromt.log file. 

In [5]:
import os

root = "wflow_test_sediment"
for path, _, files in os.walk(root):
    print(path)
    for name in files:
        if name.endswith(".xml"):
            continue
        print(f" - {name}")

wflow_test_sediment
 - hydromt.log
wflow_test_sediment/staticgeoms
wflow_test_sediment/instate
wflow_test_sediment/run_default


### Visualize and/or inspect model schematization

You can (copy and) adapt the following example notebooks in order to visualize your Wflow sediment model: 

* The **wflow plot** example notebook contains scripts to visualize your model
* The **wflow nc to raster files** example notebook contains scripts to write the nc model files to a netcdf which can be used to inspect (and modify) the model layers in e.g. QGIS.

### Add Wflow sediment layers to an existing Wflow model

If you already have a hydrological **Wflow** model and you want to extend it in order to include **sediment** as well, then you do not need to build the wflow_sediment from scratch. You can instead **update** the wflow model with the additional components needed by wflow_sediment. 

These components are available in the template **wflow_extend_sediment.ini**:

In [6]:
fn_ini = "wflow_extend_sediment.ini"
with open(fn_ini, "r") as f:
    txt = f.read()
print(txt)

[global]
data_libs        = []               # add optional paths to data yml files

[setup_config]
# model.dolake = True               # uncomment if there are lakes in the wflow model
# model.doreservoir = True          # uncomment if there are reservoirs in the wflow model

[setup_riverbedsed]
bedsed_mapping_fn   = None             # path to a mapping csv file from streamorder to river bed particles characteristics

[setup_lulcmaps]
lulc_fn          = globcover        # source for lulc maps: {globcover, vito, corine}

[setup_canopymaps]
canopy_fn        = simard           # source for vegetation canopy height: {simard}

[setup_soilmaps]
soil_fn          = soilgrids        # source for soilmaps: {soilgrids}
usleK_method     = renard           # method to compute the USLE K factor: {renard, epic}

[setup_gauges]
gauges_fn        = grdc             # If not None add gaugemap. Either a path or known gauges_fn: {grdc}
snap_to_river    = True             # If True snaps gauges from source

Let's update the hydrological **Wflow** model available from our example for the Piave subbasin:

In [7]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt update wflow_sediment "wflow_piave_subbasin" -o "./wflow_test_extend_sediment" -i wflow_extend_sediment.ini -vv

2022-01-27 12:40:49,955 - update - log - DEBUG - Writing log messages to new file /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_extend_sediment/hydromt.log.
2022-01-27 12:40:49,955 - update - log - INFO - HydroMT version: 0.4.5.dev
2022-01-27 12:40:49,955 - update - main - INFO - Updating wflow_sediment model at /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_piave_subbasin (r).
2022-01-27 12:40:49,955 - update - main - INFO - Output dir: /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_extend_sediment
2022-01-27 12:40:49,955 - update - main - INFO - User settings:
2022-01-27 12:40:49,961 - update - model_plugins - DEBUG - Loaded model plugin 'wflow_sediment = hydromt_wflow.wflow_sediment.WflowSedimentModel' (0.1.4.dev0)
2022-01-27 12:40:49,961 - update - model_api - INFO - Initializing wflow_sediment model from hydromt_wflow (v0.1.4.dev0).
2022-01-27 12:40:49,962 - update - log - DEBUG - Writin

2022-01-27 12:40:50,049 - update - wflow - INFO - Reading model intbl files.
2022-01-27 12:40:50,050 - update - model_api - ERROR - Model config file not found at /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_piave_subbasin/wflow_sediment.toml
2022-01-27 12:40:50,050 - update - wflow - INFO - Reading model staticgeom files.


2022-01-27 12:40:50,121 - update - model_api - ERROR - Model config file not found at /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_piave_subbasin/wflow_sediment.toml
2022-01-27 12:40:50,121 - update - wflow - INFO - Read forcing from /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_piave_subbasin/inmaps.nc
2022-01-27 12:40:50,130 - update - wflow - INFO - Model read
2022-01-27 12:40:50,131 - update - log - DEBUG - Appending log messages to file /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_extend_sediment/hydromt.log.
2022-01-27 12:40:50,134 - update - model_api - INFO - setup_riverbedsed.bedsed_mapping_fn: None
2022-01-27 12:40:50,134 - update - wflow_sediment - INFO - Preparing riverbedsed parameter maps.
2022-01-27 12:40:50,146 - update - landuse - INFO - Deriving D50_River using average resampling (nodata=-999.0).


2022-01-27 12:40:50,155 - update - landuse - INFO - Deriving ClayF_River using average resampling (nodata=-999.0).
2022-01-27 12:40:50,165 - update - landuse - INFO - Deriving SiltF_River using average resampling (nodata=-999.0).
2022-01-27 12:40:50,174 - update - landuse - INFO - Deriving SandF_River using average resampling (nodata=-999.0).
2022-01-27 12:40:50,183 - update - landuse - INFO - Deriving GravelF_River using average resampling (nodata=-999.0).


2022-01-27 12:40:50,206 - update - model_api - INFO - setup_lulcmaps.lulc_fn: globcover
2022-01-27 12:40:50,206 - update - model_api - INFO - setup_lulcmaps.lulc_mapping_fn: None
2022-01-27 12:40:50,206 - update - model_api - INFO - setup_lulcmaps.lulc_vars: ['landuse', 'Cov_River', 'Kext', 'N', 'PathFrac', 'Sl', 'Swood', 'USLE_C', 'WaterFrac']
2022-01-27 12:40:50,207 - update - wflow - INFO - Preparing LULC parameter maps.
2022-01-27 12:40:50,210 - update - data_adapter - INFO - Adding sample data v0.0.6 from artifacts


...

2022-01-27 12:40:52,350 - update - wflow - INFO - Gauges map based on catchment river outlets added.
2022-01-27 12:40:52,350 - update - data_adapter - INFO - DataCatalog: Getting grdc GeoDataFrame csv data from /home/runner/.hydromt_data/data/v0.0.6/grdc.csv
2022-01-27 12:40:52,353 - update - data_adapter - INFO - GeoDataFrame: Read csv data and clip to geom (epsg:4326) [11.775, 45.804, 12.745, 46.690].
2022-01-27 12:40:52,366 - update - wflow - INFO - 3 grdc gauge locations found within domain


2022-01-27 12:40:53,263 - update - model_api - DEBUG - Default config read from /usr/share/miniconda/envs/hydromt-wflow/lib/python3.9/site-packages/hydromt_wflow/data/wflow_sediment/wflow_sediment.toml
2022-01-27 12:40:53,264 - update - wflow - INFO - Gauges map from grdc added.
2022-01-27 12:40:53,287 - update - wflow - INFO - Write model data to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_extend_sediment
2022-01-27 12:40:53,291 - update - model_api - INFO - Writing model config to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_extend_sediment/wflow_sediment.toml
2022-01-27 12:40:53,292 - update - wflow - INFO - Write staticmaps to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_extend_sediment/staticmaps.nc


2022-01-27 12:40:53,835 - update - wflow - INFO - Writing model staticgeom to file.


2022-01-27 12:40:53,981 - update - wflow - INFO - Write forcing file
2022-01-27 12:40:53,992 - update - wflow - INFO - Process forcing; saving to /home/runner/work/hydromt_wflow/hydromt_wflow/docs/examples/examples/wflow_test_extend_sediment/inmaps.nc
[                                        ] | 0% Completed |  0.0s

[########################################] | 100% Completed |  0.1s


The example above means the following: run **hydromt update** with:

* `wflow_sediment` : i.e. update a wflow_sediment model (in our case a wflow model but with wflow_sediment components)
* `wflow_piave_subbasin` : hydrological wflow model folder
* `-o "./wflow_test_extend_sediment"`: output combined wflow hydrology+sediment models
* `-i wflow_extend_sediment.ini` : setup configuration file containing wflow sediment specific components
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.